In [2]:
# What are Static and Class methods
# Static methods: does not depend on instance or class attributes. Utility functions or calculations.
# Class methods : Operate on class level attributes. Often used for factory methods. Used for factory or shared state.

class Calculator:
    base_value = 100

    @staticmethod
    def add(value1, value2):
        return value1 + value2
    
    @classmethod
    def multiply_base(cls, multiplier):
        return cls.base_value * multiplier
    
# Using Static Method
print(Calculator.add(4,5))

#Using Class Method
print(Calculator.multiply_base(2))

9
200


In [3]:
class Utility:
    @staticmethod
    def greet_user(name):
        print(f"Hello, {name}")

Utility.greet_user("John")

Hello, John


In [4]:
class Counter:
    count = 0

    @classmethod
    def increment(cls):
        cls.count += 1

Counter.increment()
print(Counter.count)

1


In [5]:
# Inventory Management System
# Bonus part: Define a date for perishable products

from datetime import datetime, timedelta

class Inventory:
    total_items = 0
    alert_quantity_threshold = 5
    alert_expiration_threshold = 3

    def __init__(self, product_name, price, quantity):
        self.product_name = product_name
        self.price = price
        self.quantity = quantity
        self.expiration_date = datetime.strptime("3999-12-31", "%Y-%m-%d")

    @classmethod
    def add_product(cls, quantity):
        cls.total_items += quantity

    # Instance Method : Alert in case of close expiration date
    def alert_expiration_date(self):
        if self.expiration_date - datetime.now() <= timedelta(days=3):
            print(f"{self.product_name} will expire in less than {Inventory.alert_expiration_threshold} days. Expiration date : {self.expiration_date}.")

    # Instance Method : Alert in case of low quantity
    def alert_quantity(self):
        if self.quantity <= Inventory.alert_quantity_threshold:
            print(f"{self.product_name} quantity is less than {Inventory.alert_quantity_threshold}. Quantity: {self.quantity}. Please consider reapprovisionning.")

    # Instance Method : Set Product expiration date
    def set_expiration_date(self, expiration_date):
        self.expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")

    # Instance Method: Show Product Details
    def show_product_details(self):
        print("\n--- Product Details ---")
        print(f"Product Name: {self.product_name}")
        print(f"Price: {self.price}")
        print(f"Quantity: {self.quantity}")
        expiration_date = datetime.strftime(self.expiration_date, "%Y-%m-%d")
        print(f"Expiration Date: {expiration_date if expiration_date != '3999-12-31' else 'NA'}")

    # Instance Method: Sell Product
    def sell_product(self, amount):
        if amount <= self.quantity:
            self.quantity -= amount
            Inventory.total_items -= amount
            print(f"{amount} {self.product_name}(s) sold.")
        else:
            print("Insufficient quantity.")

    # Static Method: Calculate Discount
    @staticmethod
    def calculate_discount(price, discount_percentage):
        return price * (1 - discount_percentage / 100)
    
    # Class Method: Total Items Report
    @classmethod
    def total_items_report(cls):
        print(f"\nTotal Items: {cls.total_items}")

# Main Program
INVENTORY_FILE = './assets/inventory.json'
import json

class InventoryManagementSystem:
    def __init__(self):
        self.products = self.load_products()
        self.trigger_alerts()
        
    # Trigger alerts
    def trigger_alerts(self):
        print("\n--- Alerts ---")
        for product in self.products:
            product.alert_expiration_date()
            product.alert_quantity()

    # Add Product to inventory
    def add_product(self):
        try:
            product_name = input("Enter product name: ")
            price =  float(input("Enter price: ").strip())
            quantity = int(input("Enter quantity: ").strip())
            product = Inventory(product_name, price, quantity)
            is_perishable = input("Is it a perishable product (yes/no) ?").lower()
            if is_perishable == "yes":
                expiration_date = input("Enter Expiration Date (YYYY-MM-DD): ").strip()
                product.set_expiration_date(expiration_date)
            Inventory.add_product(product.quantity)
            self.products.append(product)
            self.save_products()
            print(f"{quantity} {product_name}(s) added to inventory.")
        except ValueError as e:
            print(f"Incorrect value : {e}")
        except Exception as e:
            print(f"An unexpected error occurred : {e}")

    # Display All products
    def view_products(self):
        print("\n--- Inventory ---")
        if not self.products:
            print("No products in inventory.")
        else:
            for product in self.products:
                product.show_product_details()

    # Sell Product
    def sell_product(self):
        product_name = input("Enter product name to sell: ").strip()
        for product in self.products:
            if product.product_name == product_name:
                try:
                    amount = int(input("Enter amount to sell: ").strip())
                    product.sell_product(amount)
                    self.save_products()
                    return
                except ValueError as e:
                    print(f"Incorrect value : {e}")
                except Exception as e:
                    print(f"An unexpected error occurred : {e}")
        print("Product not found in inventory.")

    # Calculate Discount
    def discount_price(self):
        try:
            price = float(input("Enter price: ").strip())
            discount_percentage = float(input("Enter discount percentage: ").strip())
            discounted_price = Inventory.calculate_discount(price, discount_percentage)
            print(f"Discounted Price: {discounted_price}")
        except ValueError as e:
            print(f"Incorrect value : {e}")
        except Exception as e:
            print(f"An unexpected error occurred : {e}")

    def convert_products_data(self):
        return [
            {
                "product_name": product.product_name,
                "price": product.price,
                "quantity": product.quantity,
                "expiration_date": datetime.strftime(product.expiration_date, "%Y-%m-%d")
            }
            for product in self.products
        ]
    
    def save_products(self):
        try:
            with open(INVENTORY_FILE, 'w') as json_file:
                json.dump(self.convert_products_data(), json_file, indent=2)
        except Exception as e:
            print(f"An unexpected error occured : {e}")

    def load_products(self):
        products = []
        try:
            Inventory.total_items = 0
            with open(INVENTORY_FILE, 'r') as json_file:
                products_data = json.load(json_file)
                if products_data:
                    for product_data in products_data:
                        Inventory.add_product(product_data["quantity"])
                        product = Inventory(product_data["product_name"], product_data["price"], product_data["quantity"])
                        product.set_expiration_date(product_data.get("expiration_date","3999-12-31"))
                        products.append(product)
        except FileNotFoundError:
            print("Starting without any product in the inventory.")
        except Exception as e:
            print(f"An unexpected error occurred : {e}")
        return products
    
# Main Menu
while True:
    inventory_ms = InventoryManagementSystem()

    print("\n--- Inventory Management System ---")
    print("1. Add Product")
    print("2. View Products")
    print("3. Sell Product")
    print("4. Calculate discount")
    print("5. Total Items Report")
    print("6. Exit")

    choice = input("Enter your choice (1-6): ")

    if choice == "1":
        inventory_ms.add_product()
    elif choice == "2":
        inventory_ms.view_products()
    elif choice == "3":
        inventory_ms.sell_product()
    elif choice == "4":
        inventory_ms.discount_price()
    elif choice == "5":
        Inventory.total_items_report()
    elif choice == "6":
        print("Exiting Inventory Management System. Goodbye !")
        break
    else:
        print("Invalid choice. Please enter a number between 1 and 6")


--- Alerts ---
Milk will expire in less than 3. Expiration date : 2025-06-28 00:00:00
Laptop quantity is less than 5. Quantity: 3. Please consider reapprovisionning.

--- Inventory Management System ---
1. Add Product
2. View Products
3. Sell Product
4. Calculate discount
5. Total Items Report
6. Exit

--- Inventory ---

--- Product Details ---
Product Name: Milk
Price: 1.0
Quantity: 100
Expiration Date: 2025-06-28

--- Product Details ---
Product Name: Laptop
Price: 1500.0
Quantity: 3
Expiration Date: NA

--- Alerts ---
Milk will expire in less than 3. Expiration date : 2025-06-28 00:00:00
Laptop quantity is less than 5. Quantity: 3. Please consider reapprovisionning.

--- Inventory Management System ---
1. Add Product
2. View Products
3. Sell Product
4. Calculate discount
5. Total Items Report
6. Exit
50 Milk(s) sold.

--- Alerts ---
Milk will expire in less than 3. Expiration date : 2025-06-28 00:00:00
Laptop quantity is less than 5. Quantity: 3. Please consider reapprovisionning.
